In [1]:
import xlrd
import numpy as np

#read class
def readTranscript(filename='genes_transcript.xlsx'):
    sheet = xlrd.open_workbook(filename).sheet_by_index(0)
    nRows = sheet.nrows
    my_class_label = []
    my_class_value = []
    for rowNum in range(0,nRows):
        rowList = []
        row = sheet.row_values(rowNum)
        rowLabel = row.pop(0)
        my_class_label.append([rowLabel])
        for el in row:
            rowList.append([el])
        rowList = np.array(rowList)
        my_class_value.append(rowList)
    my_class_label = np.array(my_class_label)
    my_class_value = np.array(my_class_value)
    #print(my_class_label)
    #print(my_class_value)
    return my_class_label, my_class_value

#read data
def readData(filename='gene_data.xlsx'):
    sheet = xlrd.open_workbook(filename).sheet_by_index(0)
    nRows = sheet.nrows
    my_data_value = []
    for rowNum in range(0,nRows):
        rowList = []
        row = sheet.row_values(rowNum)
        for el in row:
            rowList.append([el])
        rowList = np.array(rowList)
        my_data_value.append(rowList)
    my_data_value = np.array(my_data_value)
    #print(my_data_value)
    return my_data_value

class_name, class_value = readTranscript()
data_value = readData()

In [2]:
#Chi square feature selection

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

#select 100 features
def chisq_feature_selection(X, y,numoffeature=100):
    #normalize data
    #mms = MinMaxScaler()
    #Xnorm = mms.fit_transform(X)
    Xnorm = X
    #chi square
    chisq = SelectKBest(chi2, k=numoffeature)
    Xchi = chisq.fit_transform(Xnorm, y)
    return Xchi


In [3]:
data = np.copy(data_value)
#print(data)

In [4]:
#remove header(name of each sample)
data1 = np.delete(data, 0,axis=0)
#print(data1)

#remove first row(ID row)
data2 = np.delete(data1, 0, axis=1)
#print(data2)

#turn 3dd list to 2d list
#print(data2.shape)
data3 = np.reshape(data2, (data2.shape[0], data2.shape[1],))
#print(data3.shape)

#transpose to match the class
data4 = data3.transpose()
#print(data4)

#change from str to float
data_float = data4.astype(np.float)
#print(data_float)

In [11]:
#select gleason (total) score
y = class_value[3]
#print(y)

#change from str to float
y_float = y.astype(np.float)
#print(y_float)

#reshape y to 1d
y_float = np.reshape(y_float, (y_float.shape[0]))


In [6]:

X = chisq_feature_selection(data_float, y_float)
print(X.shape)

(494, 100)


In [12]:
#train test split
#from sklearn.model_selection import train_test_split
#train_data, test_data, train_class, test_class = train_test_split(X, y_float, test_size=0.1, random_state=20201217)

#kfold, no shuffle to get the same result every time
#kfold = KFold(n_splits=10, shuffle=False)
#for train_index, test_index in kfold.split(train_data)
print(y_float.shape)

(494,)


In [13]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

svmClassifier = svm.SVC()
scores = cross_val_score(svmClassifier, X, y_float, cv=10)
print(scores)

C:\Users\yahagi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


[0.42       0.48       0.48       0.48       0.51020408 0.53061224
 0.48979592 0.51020408 0.46938776 0.55102041]


In [ ]:
#cross val with chi square
import warnings
from matplotlib import pyplot as plt
max_feature = 600
svmClassifier = svm.SVC()

#disable warning
warnings.filterwarnings(action='ignore')

scoreList = []
for fea in range(1, max_feature+1):
    Xfea = chisq_feature_selection(data_float, y_float,fea)
    
    scores = cross_val_score(svmClassifier, X, y_float, cv=10)
    scoreList.append(scores[-1])


In [ ]:
#plot result
#print(scoreList)
plt.figure()
plt.title('cross val score under different number of features')
plt.plot(range(1,max_feature+1),scoreList )
plt.show()

In [ ]:
#mutual information feature selection
from sklearn.feature_selection import mutual_info_classif
#select 100 features
def mutualInfo_feature_selection(X, y,numoffeature=100):
    #normalize data
    #mms = MinMaxScaler()
    #Xnorm = mms.fit_transform(X)
    Xnorm = X
    #chi square
    mutualInfo = SelectKBest(mutual_info_classif, k=numoffeature)
    Xmutual = mutualInfo.fit_transform(Xnorm, y)
    return Xmutual

In [ ]:
#cross val with mutual info
import warnings
from matplotlib import pyplot as plt
max_feature = 600
svmClassifier = svm.SVC()

#disable warning
warnings.filterwarnings(action='ignore')

scoreList = []
for fea in range(1, max_feature+1):
    Xfea = mutualInfo_feature_selection(data_float, y_float, fea)
    
    scores = cross_val_score(svmClassifier, X, y_float, cv=10)
    scoreList.append(scores[-1])

In [ ]:
#f test feature selection
from sklearn.feature_selection import f_classif
#select 100 features
def f_test_feature_selection(X, y,numoffeature=100):
    #normalize data
    #mms = MinMaxScaler()
    #Xnorm = mms.fit_transform(X)
    Xnorm = X
    #chi square
    f_test = SelectKBest(f_classif, k=numoffeature)
    Xf = f_test.fit_transform(Xnorm, y)
    return Xf

In [ ]:
#cross val with f test
import warnings
from matplotlib import pyplot as plt
max_feature = 600
svmClassifier = svm.SVC()

#disable warning
warnings.filterwarnings(action='ignore')

scoreList = []
for fea in range(1, max_feature+1):
    Xfea = f_test_feature_selection(data_float, y_float, fea)
    
    scores = cross_val_score(svmClassifier, X, y_float, cv=10)
    scoreList.append(scores[-1])